### get the environment and import packages

In [1]:
import dotenv,os
import numpy as np
import pandas as pd
from langchain.chains import LLMChain
from langchain_community.llms import ChatGLM
from langchain_core.prompts import PromptTemplate

dotenv.load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")


- 生成一个列表，存储所有的可修复的产品的wiki标题，以及对于其内容的链接


In [2]:
guide_dir = r"/root/autodl-tmp/fix-helper/data/Guide"
wiki_dir = r"/root/autodl-tmp/fix-helper/data/Wiki"

In [3]:
from langchain.prompts import PromptTemplate
system_msg = """
You should simulate a customer's question whose device is broken, and an assistant's answer who help with repair device,
for each input:
firstly, simulate customer's question, make it informal, More divergent generation
secondly, You need to based on the information and the question you generated just now to simulate the assitant's answer

in every input , You will get a batch of repair methods or repair instructions from the repairman. 

Always Output in standard JSON parsed string , have 'question' in string and 'answer' in string :
Before each output, check whether you have completed the above requirements
"""
prompt = """
I will give you a instruction and you should generate thing depend on it
These instructions will be placed in the delimited symbols like ```` 
the instructions have a topic and content, I will give in below:

````topic : {topic}````
````content : {content}````

Always Output in standard JSON parsed string , have 'question' in string and 'answer' in string :
Before each output, check whether you have completed the above requirements
"""

In [4]:
# import time
# import timeout_decorator
# from func_timeout import func_set_timeout
from openai import OpenAI
import openai
import time

client = OpenAI(base_url='https://api.kwwai.top/v1', api_key='sk-VtFAumQUfwPCQJxe27379c076fE64e99B52e9078C87f6eD3')

def get_completion_messages(text):
    return [  
        {
            'role':'system', 
            'content': system_msg
        },    
        {
            'role':'user', 
            'content': text
        },  
    ]

# @timeout_decorator.timeout(65)  # 设置超时时间为65秒

# @func_set_timeout(65)
def get_completion_from_messages(
    messages, 
    model="gpt-3.5-turbo", 
    temperature=1.2, 
    max_tokens=3000,
    retries = 3,
):
    retry_cnt = 0
    backoff_time = 30
    while retry_cnt <= retries:
        try:
            response = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=temperature, 
                max_tokens=max_tokens, 
            )
            return response.choices[0].message.content
        except:
            # print(f"OpenAIError: {e}.")
            # if "Please reduce your prompt" in str(e):
            #     max_tokens = int(max_tokens * 0.8)
            #     print(f"Reducing target length to {max_tokens}, retrying...")
            # else:
            #     print(f"Retrying in {backoff_time} seconds...")
            #     time.sleep(backoff_time)
            #     backoff_time *= 1.5
            retry_cnt += 1

    

In [5]:
# json_in_file = json.dumps(json_data,ensure_ascii=False, indent=4)
# with open('generate_file.json','w',encoding='utf-8') as file:
#     file.write(json_in_file) 

In [6]:
import json
guide_files = os.listdir(guide_dir)
wiki_files = os.listdir(wiki_dir)
guide_root = r"/root/autodl-tmp/fix-helper/data/Guide"
wiki_root = r"/root/autodl-tmp/fix-helper/data/Wiki"
generate_QA = []

In [7]:
import random
import urllib.parse
from tqdm import tqdm


visited_file = set()

def random_select_file(file_list):
    select_file = random.choice(file_list)
    while select_file in visited_file:
        select_file = random.choice(file_list)
    visited_file.add(select_file)
    return select_file

def get_single_completion(file_list,file_root):
    file_src = random_select_file(file_list)
    file_url = file_root + '/' + file_src
    # print(file_url)
    file_topic = file_src.split('.')[0]
    file_topic = urllib.parse.unquote(file_topic)
    try:
        with open(file_url,'r',encoding='utf-8') as file:
            file_content = file.read()
    except:
        return None
    prompt_full = PromptTemplate.from_template(prompt).format_prompt(topic=file_topic,content=file_content)
    message = get_completion_messages(prompt_full.text)
    # print(message)
    response = get_completion_from_messages(message)
    return response

def add_to_QA(file_list,file_root):
    message = get_single_completion(file_list,file_root)
    if message is None:
        return
    try:
        item_one = json.loads(message)
    except:
        return
    generate_QA.append(item_one)

def do_generate():
    for i in tqdm(range(80)):
        add_to_QA(guide_files,guide_root)
    for i in tqdm(range(20)):
        add_to_QA(wiki_files,wiki_root)

# do_generate()

In [8]:
# response = get_single_completion(guide_files,guide_root)
# print(response)

In [9]:
do_generate()
json_in_file = json.dumps(generate_QA,ensure_ascii=False, indent=4)
with open('generate_file.json','w',encoding='utf-8') as file:
    file.write(json_in_file) 

  0%|          | 0/80 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:51<00:00,  2.55s/it]


In [10]:
json_in_file = json.dumps(generate_QA,ensure_ascii=False, indent=4)
with open('generate_file2.json','w',encoding='utf-8') as file:
    file.write(json_in_file) 